In [1]:
import re
import os
import pandas as pd
import numpy as np
import json

In [2]:
common_tids = open("cm_tid.json")

In [3]:
common_tids = json.load(common_tids)

In [4]:
common_tids

{'0': 1566,
 '1': 4354,
 '2': 5192,
 '3': 6547,
 '4': 18546,
 '5': 36875,
 '6': 36937,
 '7': 37934,
 '8': 37963,
 '9': 37987,
 '10': 39679,
 '11': 49480,
 '12': 53080,
 '13': 64332,
 '14': 67255,
 '15': 68469,
 '16': 71315,
 '17': 74028,
 '18': 75646,
 '19': 79561,
 '20': 82620,
 '21': 83406,
 '22': 88397,
 '23': 89164,
 '24': 89564,
 '25': 91088,
 '26': 91635,
 '27': 94514,
 '28': 96002,
 '29': 96706,
 '30': 96932,
 '31': 100123,
 '32': 102852,
 '33': 104377,
 '34': 120358,
 '35': 128161,
 '36': 130087,
 '37': 135809,
 '38': 135953,
 '39': 137050,
 '40': 138184,
 '41': 146267,
 '42': 150473,
 '43': 152460,
 '44': 161939,
 '45': 162420,
 '46': 170697,
 '47': 174974,
 '48': 175829,
 '49': 181033,
 '50': 187706,
 '51': 188348,
 '52': 189907,
 '53': 195994,
 '54': 199977,
 '55': 206804,
 '56': 211970,
 '57': 212274,
 '58': 213382,
 '59': 214713,
 '60': 220229,
 '61': 222788,
 '62': 223504,
 '63': 227988,
 '64': 231341,
 '65': 241970,
 '66': 243002,
 '67': 246887,
 '68': 248715,
 '69': 256

In [10]:
common_tids_arr = []
for i in common_tids.keys():
    common_tids_arr.append(common_tids[i])

In [12]:
tids_to_be_indexed = []

In [5]:
stanza_ner = os.chdir("STANZA_TRIPLES_OLD/SIMILARITY_TRIPLES")

In [13]:
for file in os.listdir():
    if file.endswith(".csv"):
        if int(file[:-4]) in common_tids_arr:
            tids_to_be_indexed.append(int(file[:-4]))

In [14]:
os.chdir("..")
os.chdir("..")

In [15]:
os.chdir("STANZA_TRIPLES_NEW")

In [16]:
for file in os.listdir():
    if file.endswith(".csv"):
        if int(file[:-4]) not in tids_to_be_indexed:
            tids_to_be_indexed.append(int(file[:-4]))

In [21]:
tids_to_consider = {}
for i in range(len(tids_to_be_indexed)):
    tids_to_consider[i] = int(tids_to_be_indexed[i])

In [22]:
with open('tids_to_consider.json','w') as fp:
    json.dump(tids_to_consider , fp , indent=4)

In [23]:
os.chdir("..")

In [26]:
f = open('tids_to_consider.json')
tids_c = json.load(f)

In [28]:
tids_c_arr = []
for i in tids_c.keys():
    tids_c_arr.append(tids_c[i])

In [67]:
stanza_ner_merged = pd.read_csv("stanza_triples_merged_csv.csv",encoding = "utf-8")

In [68]:
stanza_ner_merged.shape[0]

652930

In [74]:
df = pd.read_csv("stanza_triples_merged_csv.csv"  , encoding = "utf-8")

Steps to perform

1. Clean the tokens from stanza_ner_merged_final_set and index them
2. Read files one by one using tids_c_arr
3. Find most frequent term in each label
4. Get the index of the most frequent term
5. Use mapping.json and add the tids not present in it
6. Use OE_LABELS file and update its rows which have tids in common with tids_c_arr
7. Pick only those files from OE_LABELS
8. Create the TRAIN_SET.csv
9. Create the TEST_SET.csv

In [75]:
df.shape[0]

652930

In [81]:
tokens = []
for i in range(df.shape[0]):
    if df.iloc[i,1] == 'EVENT' or df.iloc[i,1] == 'FAC' or df.iloc[i,1] == 'WORK_OF_ART' or df.iloc[i,1] == 'CARDINAL' or df.iloc[i,1] == 'DATE':
        continue
    else:
        tokens.append(df.iloc[i,2])

In [ ]:
#now token contains labels of PERSON, LOC, LAW, ORG, GPE, NORP, PERCENT, LANGUAGE

In [82]:
len(tokens) , tokens

(424426,
 ['Delhi High Court',
  'M/S. Seemax Construction (P) Ltd.',
  'State Bank Of India And Another',
  'Y Sabharwal',
  'No. 2',
  'Plaintiff',
  'No. I State Bank of\nIndia',
  'Rs. 5 lakhs',
  'Rr. 1 and\n2',
  'S. 151',
  'Bikaner',
  '1,50,000',
  '1,50,0001',
  'No. 2',
  'Pune',
  'Rs. 5 lakhs',
  'Rs. 5 lakhs',
  'Pune',
  'No. 2',
  'Rs. 5 lakhs',
  'No. 2',
  'No. 2',
  'No. 2',
  'Plaintiff',
  '3-7-91',
  'Plaintiff',
  '10-7-91',
  'Rs. 5 lakhs',
  'Bikaner',
  'Rajasthan',
  'Bikaner',
  'Delhi',
  'Delhi',
  'Delhi',
  'Court',
  'No. 2',
  'Bikaner',
  'S. 101 of the Multi State Cooperative\nAct',
  'No. 2',
  'No. 2',
  'Bikaner',
  'the Bikaner Project',
  'Shri Ganganagar Project',
  'Bikaner',
  '156 lakhs',
  'Bikaner',
  'Bikaner',
  'Pune',
  'Shri Ganganagar',
  'Jaipur',
  'Shri Ganganagar',
  'Bikaner',
  'Rs. 10 lakhs',
  'the World Bank',
  'European Economic Community',
  'No. 2',
  'Court',
  'Bikaner',
  '5 lakhs',
  'Bikaner',
  'Plaintiff',
  'Bika

In [78]:
#clean the tokens

In [83]:
for i in range(len(tokens)):
    tokens[i] = clean_token(tokens[i])

In [84]:
len(tokens) , tokens

(424426,
 ['delhi high court',
  'm/s. seemax construction (p) ltd.',
  'state bank of india and another',
  'y sabharwal',
  'no. 2',
  'plaintiff',
  'no. i state bank of india',
  'rs. 5 lakhs',
  'rr. 1 and 2',
  's. 151',
  'bikaner',
  '1,50,000',
  '1,50,0001',
  'no. 2',
  'pune',
  'rs. 5 lakhs',
  'rs. 5 lakhs',
  'pune',
  'no. 2',
  'rs. 5 lakhs',
  'no. 2',
  'no. 2',
  'no. 2',
  'plaintiff',
  '3-7-91',
  'plaintiff',
  '10-7-91',
  'rs. 5 lakhs',
  'bikaner',
  'rajasthan',
  'bikaner',
  'delhi',
  'delhi',
  'delhi',
  'court',
  'no. 2',
  'bikaner',
  's. 101 of the multi state cooperative act',
  'no. 2',
  'no. 2',
  'bikaner',
  'the bikaner project',
  'shri ganganagar project',
  'bikaner',
  '156 lakhs',
  'bikaner',
  'bikaner',
  'pune',
  'shri ganganagar',
  'jaipur',
  'shri ganganagar',
  'bikaner',
  'rs. 10 lakhs',
  'the world bank',
  'european economic community',
  'no. 2',
  'court',
  'bikaner',
  '5 lakhs',
  'bikaner',
  'plaintiff',
  'bikaner

In [35]:
def clean_token(token):
    #remove extra spaces
    token = re.sub(r'[ \n\t]+',' ',token)
    
    #convert to lowercase
    s = u''+str(token)+'';
    token = s.lower()
    
    return token

In [85]:
#After removing spaces and converting to lowercase

In [87]:
len(tokens)

424426

In [88]:
tokens_unique = list(set(tokens))

In [100]:
len(tokens_unique)

100302

In [92]:
tokens_dictionary = {}
for i in range(len(tokens_unique)):
    tokens_dictionary[i] = tokens_unique[i]

with open("tokens_dictionary" , "w") as of:
    json.dump(tokens_dictionary , of , indent =4)

#now read line one by one from OE_LABELS

check if tid exists in common_tid
    then read csv triples file
    for each label:
        get max count of token
        store the index of token in OE_LABELS corresponding column
repeat for new files

In [93]:
#Update Mapping.json file
#Update Train.csv using casesimilarity2020 file (similar cases)
#Update Test.csv using new judgements

In [96]:
oe_label = pd.read_csv("oe_labels.csv" , encoding = "utf-8")

In [98]:
oe_label.head()

,tid,LEGAL_SCJUDGE,LEGAL_DESIGN,LEGAL_GEN,LEGAL_PATENT,LEGAL_TRADEMARK,LEGAL_COPYRIGHT,LEGAL_RESERVATION_CATEGORY,LEGAL_CRIMINAL_PROCEDURE,LEGAL_ENTITY,...,ORGANIZATION,ORGANIZATION2,GEOGRAPHY2,DATETIME,PERCENT,COPYRIGHT_ACT,PATENTS_ACT,DESIGNS_ACT,INFORMATION_TECHNOLOGY_ACT,TRADEMARK_ACT
0,928322,0,5,21,0,0,0,4,4,7,...,174,81,246,286,3,0,0,0,0,0
1,699319,20,8,15,0,0,0,4,4,1,...,205,50,66,230,3,0,0,0,0,0
2,1988649,7,3,15,0,0,0,4,4,7,...,48,50,0,282,3,0,0,0,0,0
3,1362442,0,5,19,0,0,0,4,4,4,...,48,50,0,125,12,0,0,0,0,0
4,357132,0,0,5,0,0,0,4,10,9,...,205,50,0,120,3,0,0,0,0,0


In [ ]:
for i in range(oe_label.shape[0]):
    #get tid
    tid = oe_label.iloc[i,0]
    
    if tid in tids_c_arr:
        #pick up csv triples file
        file = pd.read_csv("STANZA_TRIPLES_MERGED/"+str(tid)+".csv", encoding="utf-8")
        labels = list(set(file['labels']))
        #create dictionary to store count
        label_count = {}
        for l in labels:
            label_count[l] = 0
        
        for l in label_count.keys():
            label_count[l] = get_max_count_token(l , tid)
        
        

In [ ]:
def get_max_count_token(label , tid):
    #read csv file of tid
    file = pd.read_csv("STANZA_TRIPLES_MERGED/"+str(tid)+".csv", encoding="utf-8")
    
    tokens_with_label = []
    for i in range(file.shape[0]):
        if file.iloc[i,1] == label:
            tokens_with_label.append(file.iloc[i,2])
    
    #clean the array
    for j in range(len(tokens_with_label)):
        tokens_with_label[j] = clean_token(tokens_with_label[j])
    
    #find most occuring token
    res = max(set(tokens_with_label), key = tokens_with_label.count)
    
    #find the index
    dict_token = 

In [99]:
d = ['a', 'ab' , 'ab' ,'c' ,'e']
max(set(d), key = d.count)

'ab'